#packages/ autres


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker

%matplotlib inline
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import re
import json
import spacy

from numpy import nan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#traitement données

## Importation des données

In [ ]:
file = '/content/drive/My Drive/Save_3.txt'

with open(file, mode='r+', encoding="utf-8") as script:
	developer = json.load(script)
script.close()
developer = re.sub(r'null','None',developer)

DF1t = pd.DataFrame.from_dict(eval(developer))
DF_ScriptPersoFilm = DF1t.T

## Prétraitement

In [ ]:
#Code de transformation des données

## Sélection des données

In [ ]:
# Code de sélection des datas
max-lenght = 1000
min-lenght = 200

#Gold-Label

## Importation Gold-Label

In [ ]:
#trouver une dataset gold label

#Pseudo Label

## Def Pseudo Label


In [ ]:
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator, RegressorMixin

class PseudoLabeler(BaseEstimator, RegressorMixin):
 '''
 Sci-kit learn wrapper for creating pseudo-lebeled estimators.
 '''

 def __init__(self, model, unlabled_data, features, target, sample_rate=0.2, seed=42):
  '''
  @sample_rate - percent of samples used as pseudo-labelled data
  from the unlabelled dataset
  '''
  assert sample_rate <= 1.0, 'Sample_rate should be between 0.0 and 1.0.'

  self.sample_rate = sample_rate
  self.seed = seed
  self.model = model
  self.model.seed = seed

  self.unlabled_data = unlabled_data
  self.features = features
  self.target = target

 def get_params(self, deep=True):
  return {
  "sample_rate": self.sample_rate,
  "seed": self.seed,
  "model": self.model,
  "unlabled_data": self.unlabled_data,
  "features": self.features,
  "target": self.target
  }

 def set_params(self, **parameters):
  for parameter, value in parameters.items():
   setattr(self, parameter, value)
  return self

 def fit(self, X, y):
  '''
  Fit the data using pseudo labeling.
  '''
  augemented_train = self.__create_augmented_train(X, y)
  self.model.fit(
   augemented_train[self.features],
   augemented_train[self.target]
  )
  return self

 def __create_augmented_train(self, X, y):
  '''
  Create and return the augmented_train set that consists
  of pseudo-labeled and labeled data.
  '''
  num_of_samples = int(len(self.unlabled_data) * self.sample_rate)

# Train the model and creat the pseudo-labels
  self.model.fit(X, y)
  pseudo_labels = self.model.predict(self.unlabled_data[self.features])

# Add the pseudo-labels to the test set
  pseudo_data = self.unlabled_data.copy(deep=True)
  pseudo_data[self.target] = pseudo_labels

# Take a subset of the test set with pseudo-labels and append in onto
 # the training set
  sampled_pseudo_data = pseudo_data.sample(n=num_of_samples)
  temp_train = pd.concat([X, y], axis=1)
  augemented_train = pd.concat([sampled_pseudo_data, temp_train])

  return shuffle(augemented_train)

 def predict(self, X):
  '''
  Returns the predicted values.
  '''
  return self.model.predict(X)

 def get_model_name(self):
  return self.model.__class__.__name__

# Deep Model

def parametres

In [ ]:
input_dim = 


def model

In [ ]:
#Simple model -> ajouter LSTM / CNN
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[train_df.shape[1]]),
    layers.Dropout(0.3, seed=2),
    layers.Dense(64, activation='swish'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='swish'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='swish'),
    layers.Dense(1)
])

In [ ]:
# exemple CNN
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split
from data_helpers import load_data

print('Loading data')
x, y, vocabulary, vocabulary_inv = load_data()

# x.shape -> (10662, 56)
# y.shape -> (10662, 2)
# len(vocabulary) -> 18765
# len(vocabulary_inv) -> 18765

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

# X_train.shape -> (8529, 56)
# y_train.shape -> (8529, 2)
# X_test.shape -> (2133, 56)
# y_test.shape -> (2133, 2)


sequence_length = x.shape[1] # 56
vocabulary_size = len(vocabulary_inv) # 18765
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 100
batch_size = 30

# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(X_test, y_test))  # starts training

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=optimizer,
              metrics=['mae'])

In [ ]:
history = model.fit(
    train_df, train_labels,
    epochs=70, validation_split=0.2
)

## Compile model



#Analyse

In [ ]:
model_history = pd.DataFrame(history.history)
model_history['epoch'] = history.epoch

fig, ax = plt.subplots(figsize=(14,8))
num_epochs = model_history.shape[0]
ax.plot(np.arange(0, num_epochs), model_history["mae"], 
        label="Training MAE", lw=3, color='#f4b400')
ax.plot(np.arange(0, num_epochs), model_history["val_mae"], 
        label="Validation MAE", lw=3, color='#0f9d58')
ax.legend()
plt.tight_layout()
plt.show()